In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
import graphing
import preprocessing
from preprocessing import remove_columns, eliminar_filas_por_valor, unique_values, filter_values, suma_columna, remove_columns_with_nulls, eliminar_nulls
from excel_formulas import rrc_unidad, reserva_de_riesgo_en_curso, calcular_devengamiento, calcular_plazo, generar_devengado, generar_rrc_unidad

In [30]:
# Replace 'your_data.xlsx' with the actual path to your Excel file
file_path = 'SINIESTRO21-22.xlsx'
template_file_path = 'ProduccionAutomovil2022-edited.xlsx'

In [31]:
# Read the excel file
df_original = pd.read_excel(file_path)

df_template = pd.read_excel(template_file_path)

In [32]:
# Remove columns that are not needed
df = remove_columns(df_original, df_template)
# df.head()

In [33]:
# Listado de productos a utilizar
valid_products = ['AUTOMOVIL- ALTA GAMA','PLAN ACCIONISTAS','REGIONAL','REGIONAL - ITAIPU / CONMEBOL','REGIONAL 0KM','REGIONAL MAX','REGIONAL PLUS','REGIONAL SUPERIOR']

# Utilizar solo los productos validos
df = filter_values(df, 'Nombre Producto', valid_products)

# Elininar filas de anulaciones
df = eliminar_filas_por_valor(df, 'Nombre Tipo Póliza', 'Anulacion')

In [34]:
# Show columns
df.columns

Index(['Nombre Tipo Póliza', 'F/Emisión', 'Fec. Desde', 'Fec. Hasta',
       'Nombre Producto', 'Fec. Desde Art.', 'Fec. Hasta Art.', 'Prima Art.'],
      dtype='object')

In [35]:
# Crear columna de Plazo
df['Plazo'] = df['Fec. Hasta Art.'] - df['Fec. Desde Art.']

In [36]:
# Show columns
df.columns
print("Longitud original:", len(df))

Longitud original: 224


In [37]:
from datetime import datetime, timedelta

# Crear variable inicio corte y fin corte en formato fecha
inicio_corte = '2022-07-01'
fin_corte = '2023-06-30'

# Asegúrate de que las columnas existan y luego crear columna de Devengado
columna1 = 'Fec. Desde Art.'
columna2 = 'Fec. Desde Art.'

if columna1 in df.columns and columna2 in df.columns:
    nuevo_df = generar_devengado(df, columna1, columna2, inicio_corte, fin_corte)
    if nuevo_df is not None:
        df = nuevo_df
else:
    print(f"Las columnas {columna1} y/o {columna2} no están presentes en el DataFrame.")

In [38]:
# Show columns
df.columns

Index(['Nombre Tipo Póliza', 'F/Emisión', 'Fec. Desde', 'Fec. Hasta',
       'Nombre Producto', 'Fec. Desde Art.', 'Fec. Hasta Art.', 'Prima Art.',
       'Plazo', 'Devengado'],
      dtype='object')

In [39]:
# Generate the "RRC Unidad" column using the function
df = generar_rrc_unidad(df, 'Plazo', 'Devengado')

# Show columns
df.columns

Index(['Nombre Tipo Póliza', 'F/Emisión', 'Fec. Desde', 'Fec. Hasta',
       'Nombre Producto', 'Fec. Desde Art.', 'Fec. Hasta Art.', 'Prima Art.',
       'Plazo', 'Devengado', 'RRC Unidad'],
      dtype='object')

In [40]:
# Convert datetime column 'Plazo' to numeric (number of days)
df['Plazo_numeric'] = df['Plazo'].dt.days

# Generate the "RRC" column using the function
df['RRC'] = reserva_de_riesgo_en_curso(df['Prima Art.'], df['Plazo_numeric'], df['RRC Unidad'])

# Drop the intermediate numeric column if you don't need it anymore
df.drop('Plazo_numeric', axis=1, inplace=True)

# Show columns
df.columns

Index(['Nombre Tipo Póliza', 'F/Emisión', 'Fec. Desde', 'Fec. Hasta',
       'Nombre Producto', 'Fec. Desde Art.', 'Fec. Hasta Art.', 'Prima Art.',
       'Plazo', 'Devengado', 'RRC Unidad', 'RRC'],
      dtype='object')

In [41]:
# Sumar la columna RRC
suma_rrc = suma_columna(df,'RRC')
print('Suma RRC: ',suma_rrc)

Suma RRC:  0.0


In [42]:
# Return excel file
df.to_excel('ResultadoProduccion.xlsx', index=False)
print("Longitud:", len(df))